# Link based ranking



## 📚 Exercise 1: Page rank  (Eigen-vector method)

In the first exercise, we implement the Page Rank algorithm using Eigen vector method.


### Goal:
1. Implementing Page rank with Eigen-vector method.
2. Testing the implemented method on a small Web with three pages (with four different link structures).

### What you are learning in this exercise:
1. Analyzing the output of PageRank for different link structures.

**Preliminaries:** If you want to normalize a vector to L1-norm or L2-norm, you can use `numpy` library as follows:

In [1]:
from __future__ import print_function, division
import numpy as np

pr = np.array([1,2,3])
print("L1-norm of {0} is {1}".format(pr, pr / np.linalg.norm(pr,1)))
print("L2-norm of {0} is {1}".format(pr, pr / np.linalg.norm(pr,2)))

L1-norm of [1 2 3] is [0.16666667 0.33333333 0.5       ]
L2-norm of [1 2 3] is [0.26726124 0.53452248 0.80178373]


**Hints:**
Please note that for this section, we are using the theoretical PageRank computation (without source of rank). See the slide "Transition Matrix for Random Walker" in the lecture notes. Columns of link matrix are from-vertex, rows of link matrix are to-vertex. We take the eigenvector with the largest eigenvalue.
We only care about final ranking of the probability vector. You can choose the normalization (or not) of your choice.

1. 注意一下返回值, The eigenvalues will be returned as an array, and the eigenvectors will be returned as a matrix where each column is an eigenvector.
2. 注意一下答案给的egienvalues的最大值是取绝对值的最大值

In [99]:
# Implement your code here
def pagerank_eigen(L, R=None):
    if R is None: # We might want to compute R outside this function to avoid recomputing large matrix
        # Construct transition probability matrix from L
        L_t = np.transpose(L)
        L_t = np.array(L_t) # 将matrix转换成array进行操作会方便点
        R_t = [item / np.linalg.norm(item,1) for item in L_t]
        R = np.transpose(R_t)
        print(L_t.shape[0])
        print(L_t.shape[1])
    # Compute eigen-vectors and eigen-values of R
    
    eigenvalues, eigenvectors = np.linalg.eig(R)
    # Take the eigen-vector with maximum eigven-value
    # Find the index of the maximum eigenvalue
    # Select the corresponding eigenvector
    
    # p = eigenvectors[:, np.argmax(eigenvalues)]
    p = np.absolute(eigenvectors[:,np.argmax(np.absolute(eigenvalues))])
    return (R,p)

np.matrix和普通的list、array一些不一样,L[0][1]不能访问matrix的元素，要用L[0,1]

In [100]:
# Test with a sample case, e.g.
L = np.matrix([
    [0,1,1], 
    [1,0,1], 
    [1,1,0]
])
R,p = pagerank_eigen(L)
print("L={0}\nR={1}\np={2}".format(L,R,p))

3
3
L=[[0 1 1]
 [1 0 1]
 [1 1 0]]
R=[[0.  0.5 0.5]
 [0.5 0.  0.5]
 [0.5 0.5 0. ]]
p=[0.57735027 0.57735027 0.57735027]


## 📚 Exercise 2: Page rank  (Iterative method)

In the second exercise, we implement the Page Rank algorithm with an iterative approach.


### Goal:
1. Implementing Page rank with the iterative method.
2. Reading the `ca-GrQc.txt` dataset and constructing its link matrix
3. Testing the implemented algorithm on the constructed link matrix
4. comparing the run-time of iterative method and eigen-vector method.

### What you are learning in this exercise:
1. Benefits of the iterative approach over eigen-vector method.

The eigen-vector method has some numerical issues (when computing eigen-vector) and not scalable with large datasets.

We will apply the iterative method in the slide "Practical Computation of PageRank" of the lecture.

Dataset for practice: https://snap.stanford.edu/data/ca-GrQc.html. It is available within the same paths of this notebook.

In [108]:
L = np.matrix([
    [0,1,1], 
    [1,0,1], 
    [1,1,0]
])
np.multiply(L, 1 / np.sum(L,axis=0))
L.shape[0]

3

In [110]:
def pagerank_iterative(L, R=None):
    if R is None:  # We might want to compute R outside this function to avoid recomputing large matrix
        R = np.multiply(L, 1 / np.sum(L,axis=0)) # multiply是elementwise的操作，等同于 *
    #todo
    # R = pagerank_eigen(L,R)
    N = L.shape[0]
    e = np.ones(shape=(N,1))
    q = 0.9

    p = e
    delta = 1
    epsilon = 0.001
    i = 0
    while delta > epsilon:
        p_prev = p
        p = np.dot(q,np.dot(R,p_prev)) 
        p = p + np.dot((1-q)/N,e)
        delta = np.linalg.norm(p - p_prev,1)
        i += 1

    print("Converged after {0} iterations. Ranking vector: p={1}".format(i, p[:,0]))
    return R,p

### Construct link matrix from dataset

In [111]:
import re
a = "1  2\n"
re.split(r'\s+', a)
a.split()
re.split(r'\s+', a)

['1', '2', '']

1. 开始无法确定L的size，所以说一定要读两次文件，第一次用来获取edge的index的一些范围，然后再确定具体内容
2. 因为L的默认是0-N，所以一定要存储对应的index值是多少

In [150]:
# Construct link matrix from file
n_nodes = 0 # number of nodes in the dataset
nodes_idx = dict() #Since the nodeIDs are not from 0 to N we need to build an index of nodes
nodes = [] #We also want to store nodeIDs to return the result of ranking vector

with open("ca-GrQc.txt") as f:
    # Fill in your code
    for line in f:
        # Check if the line starts with "#"
        if not line.startswith("#"):
            from_idx, to_idx = line.split() #虽然返回的是list，但是只要是个数对应，还是可以直接赋值的
            from_idx, to_idx = int(from_idx), int(to_idx)
            if from_idx not in nodes_idx:
                nodes_idx[from_idx] = n_nodes
                nodes.append(from_idx)
                n_nodes += 1
            if to_idx not in nodes:
                nodes_idx[to_idx] = n_nodes
                n_nodes += 1
                nodes.append(to_idx)

print(n_nodes)

L = np.zeros((n_nodes,n_nodes))

with open("ca-GrQc.txt") as f:
    # Fill in your code
    for line in f:
        # Check if the line starts with "#"
        if not line.startswith("#"):
            from_idx, to_idx = line.split() #虽然返回的是list，但是只要是个数对应，还是可以直接赋值的
            from_idx, to_idx = int(from_idx), int(to_idx)
            L[nodes_idx[from_idx],nodes_idx[to_idx]] = 1

print(nodes[:3])
            

5242
[3466, 937, 5233]


## Comparing the run-time of eigen-vector and iterative methods

### You will see that eigen-vector method is slow and has some numerical issues


#### First we pre-compute the R matrix and pass it as argument to Page rank algorithms.

In [113]:
R = np.multiply(L, 1 / np.sum(L,axis=0))  # note that this doesn't handle zero columns
print(R)

[[0.    0.2   0.5   ... 0.    0.    0.   ]
 [0.125 0.    0.    ... 0.    0.    0.   ]
 [0.125 0.    0.    ... 0.    0.    0.   ]
 ...
 [0.    0.    0.    ... 0.    0.5   0.5  ]
 [0.    0.    0.    ... 0.5   0.    0.5  ]
 [0.    0.    0.    ... 0.5   0.5   0.   ]]


In [106]:
import time
start_time = time.time()
# Run PageRank
R,p = pagerank_eigen(L, R)
print("--- %s seconds ---" % (time.time() - start_time))
print("Ranking vector: p={0}".format(p))

--- 56.280269145965576 seconds ---
Ranking vector: p=[1.85653750e-18 7.30886398e-18 1.32051427e-18 ... 0.00000000e+00
 0.00000000e+00 0.00000000e+00]


### Iterative method

In [114]:
start_time = time.time()
# Run PageRank
R, p = pagerank_iterative(L, R)
print("--- %s seconds ---" % (time.time() - start_time))
print("Ranking vector: p={0}".format(p[:,0]))

Converged after 128 iterations. Ranking vector: p=[2.91315639e-04 1.88382754e-04 8.39741651e-05 ... 1.92156702e-04
 1.92156702e-04 1.92156702e-04]
--- 1.4321448802947998 seconds ---
Ranking vector: p=[2.91315639e-04 1.88382754e-04 8.39741651e-05 ... 1.92156702e-04
 1.92156702e-04 1.92156702e-04]


## Now let's extract top-k nodes from the ranking vector

In [118]:
p.shape

(5242, 1)

In [120]:
arr = np.array(p[:,0])
k = 3
#todo index of top-k nodes
k_idx = np.argsort(arr)[::-1][:k] #注意argmax只取了最大值，而且argsort是从小到大
print("Top-{0} nodes: {1}".format(k, np.array(nodes)[k_idx]))
print("Their scores: {0}".format(arr[k_idx]))

Top-3 nodes: [14265 13801 13929]
Their scores: [0.00144951 0.00141553 0.00138011]


## 📚 Exercise 3: Ranking Methodology  (Hard)
This exercise is theoretical and you don't need to write code for it:

1. Give a directed graph, as small as possible, satisfying all the properties mentioned below:

    1. There exists a path from node i to node j for all nodes i,j in the directed
graph. Recall, with this property the jump to an arbitrary node in PageRank
is not required, so that you can set q = 1 (refer lecture slides).

    2. HITS authority ranking and PageRank ranking of the graph nodes are different.

2. Give intuition/methodology on how you constructed such a directed graph with
the properties described in (a).

3. Are there specific graph structures with arbitrarily large instances where PageRank
ranking and HITS authority ranking are the same?

## 📚 Exercise 4: Hub and Authority

In the last exercise, we implement the (iterative) HITS algorithm to calculate the authority and hub scores for this graph.


### Goal:
1. Implementing HITS algorithm.
2. Identifying the best authority and hub nodes for a small graph (part `a`) as well as the `ca-GrQc.txt` dataset (part `b`)


### a)

Let the adjacency matrix for a graph of four vertices ($n_1$ to $n_4$) be
as follows:

$
A =
  \begin{bmatrix}
	0 & 1 & 1 & 1  \\
	0 & 0 & 1 & 1 \\
	1 & 0 & 0 & 1 \\
	0 & 0 & 0 & 1 \\
  \end{bmatrix}
$

Calculate the authority and hub scores for this graph using the
HITS algorithm with k = 6, and identify the best authority and
hub nodes. (A non-zero element $A_{ij}$ indicates an edge from $i$ to $j$.)

### b)
Apply the HITS algorithm to the `ca-GrQc.txt` dataset (the same dataset as for the `Exercise 2`)

**Hint:** We follow the slide "HITS algorithm" in the lecture. **Denote $x$ as authority vector and $y$ as hub vector**. You can use matrix multiplication for the update steps in the slide "Convergence of HITS". Note that rows of adjacency matrix is from-vertex and columns of adjacency matrix is to-vertex.

In [122]:
2 ** 0.5

1.4142135623730951

这是最原始最蠢的方法,

In [130]:
# You can implement your code following this template.
def hits_iterative(A, k=10):
    N = A.shape[0]
    x0, y0 = 1 / (N**0.5) * np.ones(N), 1 / (N**0.5) * np.ones(N)
    xprev, yprev = x0, y0
    
    # For advanced exercise: define a convergence condition instead of k iterations
    for l in range(0,k):
        x = xprev
        y = yprev
        x = [np.dot(y,col_A) for col_A in np.transpose(A)]
        y = [np.dot(x,row_A) for row_A in np.array(A)]
        xprev = x/np.linalg.norm(x,2)
        yprev = y/np.linalg.norm(y,2)
        
    return xprev, yprev

1. 有多个条件就用while循环
2. while记得要给l+1

In [155]:
# You can implement your code following this template.
def hits_iterative(A, k=10):
    N = A.shape[0]
    x0, y0 = 1 / (N**0.5) * np.ones(N), 1 / (N**0.5) * np.ones(N)
    xprev, yprev = x0, y0
    delta1 = delta2 = 1
    epsilon = 0.001 # We can strictly check for convergence rate of HITS algorithm
    l = 0
    # For advanced exercise: define a convergence condition instead of k iterations
    while l < k and delta1 > epsilon and delta2 > epsilon: #  循环别忘记增加l啊
        #x = np.matmul(yprev,A)
        y = np.transpose(np.matmul(A,xprev))
        x = np.matmul(y,A)
        # y = np.matmul(A,x) 甚至上面都不用加transpose
        x = x/np.linalg.norm(x,2)
        delta1 = np.linalg.norm(x-xprev,1)
        y = y/np.linalg.norm(y,2)
        delta2 = np.linalg.norm(y-yprev,1)
        xprev = x
        yprev = y
        l += 1
    print("Ran a total of {0} iterations with the convergence rate delta1, delta2={1},{2}".format(l, delta1, delta2))
    return xprev, yprev

### Part `a` dataset

In [149]:
A=np.array([
    [0, 1, 1, 1], 
    [0, 0, 1, 1], 
    [1, 0, 0, 1],
    [0, 0, 0, 1],
])
x, y = hits_iterative(A, k=6)
print("Result using iterative method:\n Authoriy vector x={0}\n Hub vector y={1}".format(x, y))

Ran a total of 6 iterations with the convergence rate delta1, delta2=0.0007253933456335127,0.0003337175283154581
Result using iterative method:
 Authoriy vector x=[0.16854458 0.27251688 0.49793945 0.8058434 ]
 Hub vector y=[0.65546229 0.5421434  0.4051733  0.33508853]


### part `b` dataset
You can reuse the link matrix $L$ (from Exercise 2) to compute the adjacency matrix $A$ of the dataset

In [156]:
A = np.transpose(L) # your code here
print(A)

[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]


Run HITS algorithm:

In [157]:
x, y = hits_iterative(A, 100)
print("Result using iterative method:\n Authoriy vector x={0}\n Hub vector y={1}".format(x, y))

Ran a total of 22 iterations with the convergence rate delta1, delta2=0.0008757061781100327,0.0010491474133458398
Result using iterative method:
 Authoriy vector x=[3.48948782e-05 1.45965227e-05 6.68131191e-06 ... 2.07766184e-61
 2.07766184e-61 2.07766184e-61]
 Hub vector y=[3.48948790e-05 1.45965251e-05 6.68131268e-06 ... 4.73879981e-60
 4.73879981e-60 4.73879981e-60]
